<img src="../assets/headline.png" alt="headline"></a>

## Preprocessing, Exercise 2.5

## SUIT: Transformation

Lecture 2, November 9th, 2022

# 1.0 Setup
If you haven't done so already, go to section 1.0 in l1_warmup.ipynb notebook.

## 1.1 Let's import some packages

In [ ]:
from statsmodels.tsa.stattools import adfuller
import pandas as pd
from utilities.self_tests import test_your_notebook
from utilities.plot_ts import plot_series, plot_two_axis

# 1.2 Explanatory Data Analysis


### Can we skip the PHIDS?
We can skip the PHID if we saw this data a few minutes ago, but we can never skip set-index!

In [ ]:
events = pd.read_csv('../data/tiktok_events.csv')
events['datetime'] = pd.to_datetime(events['datetime'])
events.set_index('datetime',inplace=True)

In [ ]:
# Let's just make sure that this is our right data...
events.head()

# 2.0 Decreasing Likes Ratio

<img src="../assets/decrease_in_likes.png" alt="decrease_in_likes"></a>


**{Q} 5 A. What is the likes ratio from Rishon in 2022-01-06? State in integers please**

In [ ]:
groupers = [pd.Grouper(freq='1D'),'city']

events.groupby(groupers).likes.apply(lambda x: sum(x) / len(x)).unstack()

In [ ]:
A = 29

**{Q} 5 B. Plot the likes ratio per city over time. Are they stationary with p_val=1%? State True / False**

In [ ]:
groupers = [pd.Grouper(freq='1D'),'city']

likes = events.groupby(groupers).likes.apply(lambda x: 100 * sum(x) / len(x)).unstack()

columns = ['rishon', 'tel-aviv', 'ramat-gan', 'kadima']

for c in columns:
    print(f"likes ratio for {c}")
    adf = adfuller(likes[c])
    if adf[0] < adf[4].get('1%'):
        print("data is stationary")
    else:
        print("data is not stationary")
    plot_series(likes[c])

In [ ]:
B = True

**{Q} 3 C. Stationary is met, let's see what is the average like ratio for Kadima? State integer please!**

In [ ]:
events.groupby('city').likes.apply(lambda x: sum(x) / len(x))

In [ ]:
C = 15

**{Q} 5 D. Plot the views ratio per city over week from May. Are they stationary with p_val=1%? State True / False**


In [ ]:
groupers = [pd.Grouper(freq='7D'),'city']

by_date_and_city = events.groupby(groupers).views.sum()
by_date = events.groupby(pd.Grouper(freq='7D')).views.sum()

views = (100 * by_date_and_city / by_date).unstack().loc['2022-05':]

columns = ['rishon', 'tel-aviv', 'ramat-gan', 'kadima']

for c in columns:
    print(f"relative views from {c}")
    adf = adfuller(views[c])
    if adf[0] < adf[4].get('1%'):
        print("data is stationary")
    else:
        print("data is not stationary")
    plot_series(views[c])

In [ ]:
D = False

## Ramat-Gan == Simpson Effect ?
While Ramat-Gan drives the lowest like ratio, its views share is increasing with time!

<img src="../assets/simpson.png" alt="simpson"></a>


In [ ]:
events['is_ramat_gan'] = (events['city'] == 'ramat-gan').map({True:"ramat-gan", False: "not ramat-gan"})

groupers = [pd.Grouper(freq='1D'),'is_ramat_gan']

by_date_and_city = events.groupby(groupers).views.sum()
by_date = events.groupby(pd.Grouper(freq='1D')).views.sum()

views = (100 * by_date_and_city / by_date).unstack()

likes = events.resample('1D').aggregate(likes=("likes", lambda x: 100 *sum(x)/len(x)))

plot_two_axis(likes["likes"], views['not ramat-gan'], views['ramat-gan'],titles=["Likes Ratio Over Time", "Views Share Over time"], fig_length=4)

In [ ]:

# del events

In [ ]:
# Run this code block to get your grade!
NOTEBOOK_CODE = "E5"
test_your_notebook(NOTEBOOK_CODE, A, B, C, D)